In [6]:
import os
import sys
import time
#import serial
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import fftconvolve
from tqdm import tqdm 

sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

print(sys.executable)

c:\users\maxime\appdata\local\programs\python\python37\python.exe


In [7]:
# a changer
th = Thymio.serial(port="\\.\COM6", refreshing_rate=0.1)

In [8]:
class kalman():
    def __init__(self,x,y,theta, prev_x = 0, prev_y= 0, prev_theta=0, dist_wheels=94 , v_coef= 0.3375):
        self.x = x 
        self.y = y
        self.theta = theta
        self.prev_x = prev_x
        self.prev_y = prev_y
        self.prev_theta = prev_theta
    
        self.dist_wheels = dist_wheels
        self.v_coef = v_coef
        
        self.d_marker = 0 # distance between markers, a definir
        
        # a definir, avec les variances des roues et autres
        self.Q = np.array([[0, 0.0, 0.0],[0.0, 0.2, 0.0],[0.0, 0.0, 0.02]]) 
                          
        # a definir, avec le noise de machine vision
        self.R = np.array([[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0]])
    
        #a completer 
    
    def wheeldisplacements(self,dt):
        v_l = th["motor.left.speed"]*self.v_coef
        v_r = th["motor.right.speed"]*self.v_coef
        
        d_l = v_l*dt 
        d_r = v_r*dt
        return d_l, d_r
    
    
    def prediction(self, P ,sigma,dt):
        
        [x_prev, y_prev, theta_prev] = P
        [d_l, d_r] = self.wheeldisplacements(dt)
        
        # vérifier l'angle par rapport au repere global
        x = prev_x + ((d_l+d_r)/2)*np.cos(prev_theta + (d_r-d_l)/(2*self.dist_wheels))
        y = prev_y + ((d_l+d_r)/2)*np.sin(prev_theta + (d_r-d_l)/(2*self.dist_wheels))
        theta = prev_theta + (d_r-d_l)/self.dist_wheels
                     
        G = np.array([[1.0, 0.0, -((d_l+d_r)/2)*np.sin(prev_theta + (d_r-d_l)/(2*self.dist_wheels))],\
                      [0.0, 1.0, ((d_l+d_r)/2)*np.cos(prev_theta + (d_r-d_l)/(2*self.dist_wheels))],\
                      [0.0, 0.0, 1.0]])
                     
        mu = [x, y, theta]
        sigma = np.dot(np.dot(G,sigma),np.transpose(G)) + self.Q
                     
        return mu, sigma
                     
    def correction(self, measurement, P, sigma):
        
        # measurement : list de x_track1, y_track1, x_track2, y_track2 VERTICALE !
        
        [x, y, theta] = P
        
        #z = h(X) + R
        #x_track1 = x
        #y_track1 = y 
        #x_track2 = x + d_marker*cos(theta)
        #y_track2 = y + d_marker*sin(theta)
                     
        # a verifier si l'angle correspond avec tout
        H = np.array([1, 0, 0],\
                     [0, 1, 0],\
                     [1, 0, -self.d_marker*np.sin(theta)],\
                     [0, 1, self.d_marker*np.cos(theta)])
        expect_measurement = np.array([x], [y], [x + self.d_marker*np.cos(theta)], [y + self.d_marker*np.sin(theta)])
                    
        #Kalman gain 
        K_help = np.linalg.inv(np.dot(np.dot(H,sigma),np.transpose(H)) + self.R)
        K = np.dot(np.dot(sigma,np.transpose(H)),K_help)
        
        #correction
        mu = [x, y, theta] + np.dot(K,(np.array(measurement) - expect_measurement))
        sigma = np.dot(np.eye(len(sigma)) - np.dot(K,H),sigma)
        
        return mu, sigma
        



In [9]:
x = 0
y = 0
theta = 0 
P = [[x, y, theta]]
sigma=np.zeros((3,3))

#dt = 0.1

th.set_var("motor.left.target", 50)
th.set_var("motor.right.target", 50)

ka = kalman(x,y,theta)
start = time.time()
t = start
time.sleep(0.1)

while time.time()-start<5 :
    
    [prev_x, prev_y, prev_theta] = P[-1]
    prev_t = t
    t = time.time()
    dt = t-prev_t
    
    [newP, sigma] = ka.prediction(P[-1],sigma,dt)
    P.append(newP)
    time.sleep(0.1)
#    X.append(x) 
#    Y.append(y)
#    THETA.append(theta)


th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print('pose is', P)
print('sigma is', sigma)

pose is [[0, 0, 0], [0.6581612131288365, -0.0017724173500499102, -0.005385954170785052], [1.9913405761513268, -0.009192284437833684, -0.005745016926146568], [3.3245203673936916, -0.017090865493698242, -0.006104080537532235], [4.75894091359377, -0.027134465644007965, -0.00789939431433982], [6.362108343686123, -0.039510944333859044, -0.007540331558978303], [7.931498877595027, -0.05500799467922633, -0.012208158506992016], [9.48394921761879, -0.07563404931126677, -0.014362535039161115], [11.019481812557647, -0.09906839052799624, -0.016157848815968698], [12.571842277956652, -0.12582603417518676, -0.01831223048428272], [14.191605928953107, -0.1583998936136982, -0.021902858037897883], [15.861904244577396, -0.19589010717811411, -0.022980046303982433], [17.565963646591452, -0.2335259245068889, -0.021184728247054082], [19.236350915685342, -0.26681782351387456, -0.018671288959523465], [21.008056929260597, -0.29767444002430804, -0.01615784967199285], [22.797554960532473, -0.32431946025706654, -0.0

Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\users\maxime\appdata\local\programs\python\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\maxime\appdata\local\programs\python\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Maxime\Documents\Basics of mobile robotics - Projet\KALMAN\Thymio.py", line 340, in do_refresh
    self.get_variables()
  File "C:\Users\Maxime\Documents\Basics of mobile robotics - Projet\KALMAN\Thymio.py", line 499, in get_variables
    self.send(msg)
  File "C:\Users\Maxime\Documents\Basics of mobile robotics - Projet\KALMAN\Thymio.py", line 456, in send
    self.io.write(msg.serialize())
  File "c:\users\maxime\appdata\local\programs\python\python37\lib\site-packages\serial\serialwin32.py", line 323, in write
    raise writeTimeoutError
serial.serialutil.SerialTimeoutException: Write timeout



In [8]:

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)